In [2]:
# scraping scp and getting data potentially for analysis or other use

In [16]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain

In [35]:
http = httplib2.Http()

page_links = []

# directory urls are in format https://scp-wiki.wikidot.com/scp-series-{NUMBER}-tales-edition for range 1-6 inclusive

# looping to get links to all articles from each of the directory pages:
for i in range(1,7):
  status, response = http.request('https://scp-wiki.wikidot.com/scp-series-{}-tales-edition'.format(i))
  # then get links within the section of page
  # may want to flag or separate out the actual scps, have it initially be a dataframe so can store lots of data info etc.
  soup = BeautifulSoup(response, 'html.parser')
  links1 = [i.find_all('a') for i in soup.find('div', {'id': 'page-content'}).find_all('li')] 
  links2 = [i.find_all('li') for i in soup.find('div', {'id': 'page-content'}).find_all('ul')]
  # flatten and get hrefs from each item in flat list of links
  # links = [i['href'] for i in list(chain.from_iterable(links))]
  
  print(links1[:5], links2[:5])
  print(len(set(str(links1))), len(set(str(links2))))


[[], [<a href="/the-list-of-wonders">The List of Wonders</a>, <a href="/portraits-of-your-father">Portraits of Your Father</a>], [<a href="/probe-h9">Probe-H9</a>, <a href="/scp-087">SCP-087</a>, <a class="footnoteref" href="javascript:;" id="footnoteref-1" onclick="WIKIDOT.page.utils.scrollToReference('footnote-1')">1</a>], [<a href="/every-waking-hour-of-every-living-day">Every Breathing Minute of Every Waking Hour</a>, <a href="/scp-231">SCP-231</a>], []] [[<li>Articles which directly star, prominently feature, or are otherwise about the SCP in question, even if it's only a single 'scene' or chapter on any given page.</li>, <li>Articles which include the SCP for only a portion of the work in question, such as the Surgeon Crabs in the Bellerverse Canon Tale: <a href="/the-list-of-wonders">The List of Wonders</a>, or the Illusory butterflies being heavily referenced/briefly appearing in the 17,000+ word Tale "<a href="/portraits-of-your-father">Portraits of Your Father</a>".</li>, <li